In [ ]:
!rm -rf data/
!rm -rf models/

In [1]:
import os
# os.makedirs('going_modular')
# magic commands with % deals with contents of a line; those with %% deals with contents of a cell

In [2]:
import torch 
import requests
import zipfile
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = Path('data/')
image_path = data_path/'pizza_steak_sushi'
if image_path.is_dir():
    print('Skipping creating')
else:
    image_path.mkdir(parents=True, exist_ok=True)
    
with open(data_path/'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(request.content)
    
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
    zip_ref.extractall(image_path)
    
train_dir = image_path/'train'
test_dir = image_path/'test'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

Skipping creating


In [3]:
%%writefile going_modular/data_setup.py 
"""
Contains functionality to create dataloader for image classification task
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloader(
    train_dir,
    test_dir,
    transform,
    batch_size,
    num_workers=NUM_WORKERS
):
    train_data = datasets.ImageFolder(train_dir, transform)
    test_data = datasets.ImageFolder(test_dir, transform)
    
    class_names = train_data.classes
    
    train_dataloader = DataLoader(train_data, batch_size, num_workers=num_workers, shuffle=True)
    test_dataloader = DataLoader(train_data, batch_size, num_workers=num_workers, pin_memory=True, shuffle=False)
    
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [4]:
from going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir, test_dir, transform, 32)

In [28]:
%%writefile going_modular/model_builder.py
"""
Contains code to create a TinyVGG model
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
        )
        
    def forward(self, x):
        return self.classifier(self.conv_block2(self.conv_block1(x)))

Overwriting going_modular/model_builder.py


In [6]:
%load_ext autoreload
%autoreload 2

from going_modular import model_builder

model = model_builder.TinyVGG(3, 10, len(class_names)).to(device)

In [2]:
%%writefile going_modular/engine.py
"""
Contains code for training and testing step and train function.
"""
from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm

def train_step(model, dataloader, loss_fn, optim, device):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        train_acc += (y == torch.softmax(pred, dim=1).argmax(dim=1)).sum().item() / len(pred)
        optim.zero_grad()
        loss.backward()
        optim.step()
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model, dataloader, loss_fn, device):
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            test_acc += (y==torch.softmax(pred, dim=1).argmax(dim=1)).sum().item() / len(pred)
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
    return test_loss, test_acc

def train(model, train_dataloader, test_dataloader, optim, device, loss_fn=torch.nn.CrossEntropyLoss, epochs=5):
    print('Starting training')
    results = {'train_loss': [],
               'train_acc': [],
               'test_loss':[],
               'test_acc':[]}
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}---------------------------------")
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optim, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)
        print(f"Epoch: {epoch} Train acc: {train_acc:.2f} Train Loss: {train_loss:.4f} Test Acc: {test_acc:.2f} Test Loss: {test_loss:.4f}")
        results['train_loss'].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    return results

Overwriting going_modular/engine.py


In [14]:
from going_modular import engine
engine.train?

In [34]:
%%writefile going_modular/utils.py
"""
Contains utility functions
"""
import torch
from pathlib import Path

def save_model(model, target_dir, model_name):
    MODEL_PATH = Path(target_dir)
    MODEL_PATH.mkdir(parents=True, exist_ok=True)
    MODEL_SAVE_PATH = MODEL_PATH/model_name
    torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

Overwriting going_modular/utils.py


In [16]:
from going_modular import utils
utils.save_model?

In [24]:
%%writefile going_modular/train.py
"""
Contains code to train the model
"""
import os
import torch
from torchvision import transforms
from timeit import default_timer as timer
from going_modular import data_setup, engine, model_builder, utils

import argparse

NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir, test_dir, transform, BATCH_SIZE)

model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names)).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

start = timer()
engine.train(model, train_dataloader, test_dataloader, optimizer, device, loss_fn, epochs=NUM_EPOCHS)
end = timer()

utils.save_model(model, target_dir='models', model_name='05_going_modular_tinyvgg.pth')

Overwriting going_modular/train.py


In [35]:
from going_modular import train
!python going_modular/train.py

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 Train acc: 0.26 Train Loss: 1.1043 Test Acc: 0.52 Test Loss: 1.0939
Epoch: 1 Train acc: 0.56 Train Loss: 1.0911 Test Acc: 0.44 Test Loss: 1.0936
Epoch: 2 Train acc: 0.39 Train Loss: 1.0863 Test Acc: 0.31 Test Loss: 1.0793
Epoch: 3 Train acc: 0.44 Train Loss: 1.0400 Test Acc: 0.37 Test Loss: 1.0492
Epoch: 4 Train acc: 0.37 Train Loss: 1.0606 Test Acc: 0.54 Test Loss: 1.0202


Traceback (most recent call last):
  File "C:\Users\InE_STD\Documents\Learning\Computer vision\tutorials\PyTorch for DL (Udemy)\going_modular\train.py", line 8, in <module>
    from going_modular import data_setup, engine, model_builder, utils
ModuleNotFoundError: No module named 'going_modular'
